## Section 2 of LLM Project - Summarisation after preprocessing earlier

**In the previous notebook** (llm_project_1_preprocess) we took a [Hugging Face Dataset](https://huggingface.co/datasets/alexfabbri/multi_news) with a large number of long-form news articles, along with a human-generated summary of the news article.
- EDA of the range of character length, removed outliers
- The training, testing and validation data was preprocessed with removing punctuation, lower case, tokenized
- Saved as csvs to preserve the steps taken

**This notebook** will explore LLMs to summarize the original Multi-News documents, and use ROUGE Score to compare the similarity to the human generated summaries in the training set. This is a sequence-to-sequence comparison.

**Start with the Google T5 model.**

As a possible phase 2 activity I'd like to see the difference in performance by feeding the tokenized text that I preprocessed myself in the previous notebook.

I was running into major RAM / Memory issues with the free-to-use Google Colab and experimented with reducing the dataset size - the original set linked above had 45,000 records in training. I will point out any 'trims' to dataset size below.

Begin with Imports

In [52]:
!pip install transformers datasets evaluate rouge_score torch

In [11]:
from datasets import load_dataset

multi_news = load_dataset("multi_news")

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

multi_news.py:   0%|          | 0.00/3.83k [00:00<?, ?B/s]

The repository for multi_news contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/multi_news.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


train.src.cleaned:   0%|          | 0.00/548M [00:00<?, ?B/s]

train.tgt:   0%|          | 0.00/58.8M [00:00<?, ?B/s]

...

test.src.cleaned:   0%|          | 0.00/69.0M [00:00<?, ?B/s]

test.tgt:   0%|          | 0.00/7.31M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5622 [00:00<?, ? examples/s]

In [12]:
multi_news

DatasetDict({
    train: Dataset({
        features: ['document', 'summary'],
        num_rows: 44972
    })
    validation: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
    test: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
})

Same Dataset as before - this is an HF dataset and previously we converted to a Pandas Dataframe and manipulated it to preprocess and tokenize.

We'll use the AutoTokenizer in transformers this time to train the model.

In [5]:
# log in to upload model

from huggingface_hub import notebook_login

notebook_login()

## RAM / Memory Issues

#### The multi-news dataset is too big (45k records in training) and causing too many issues running in Google Colab.

#### I am going to use train_test_split on the three sections to retain only **25% of the original dataset for each**

In [13]:
from datasets import DatasetDict

# reduce train set using train_test_split, keeping only the train section
train_reduced = multi_news['train'].train_test_split(test_size=0.75, seed=42)['train']

# reduce test set using train_test_split, keeping only the train section
test_reduced = multi_news['test'].train_test_split(test_size=0.75, seed=42)['train']

# reduce validation set in the same way
val_reduced = multi_news['validation'].train_test_split(test_size=0.75, seed=42)['train']

# create a new DatasetDict with the reduced sets - news_small
news_small = DatasetDict({
    'train': train_reduced,
    'test': test_reduced,
    'validation': val_reduced
})

In [14]:
news_small

DatasetDict({
    train: Dataset({
        features: ['document', 'summary'],
        num_rows: 11243
    })
    test: Dataset({
        features: ['document', 'summary'],
        num_rows: 1405
    })
    validation: Dataset({
        features: ['document', 'summary'],
        num_rows: 1405
    })
})

In [15]:
news_small['train'][0]

{'document': 'New Zealand is wild. \n \n Just ask this Kiwi kayaker, Kyle Mulinder, who happened to be in “the wrong place at the wrong time” to get slapped directly in the face by an octopus wielded by a seal. \n \n Yes, you read that right: a seal had an octopus in its mouth in the midst of a battle royale of marine creatures. The seal emerged from the water flailing around with the octopus, which ended up hitting the unsuspecting kayaker in the face while he was out enjoying the great outdoors. \n \n “We were just sitting out in the middle of the ocean and then this huge male seal appeared with an octopus and he was thrashing him about for ages,” Mulinder told Yahoo News about the freak encounter with nature. Then came the big moment. “I was like ‘mate, what just happened?’” As it turns out, Mulinder was out kayaking with a team of content creators for GoPro testing camera equipment — so they were able to capture all the footage of the incident on video. \n \n Mulinder appears to be

Above: Confirm same structure, 'document' is the feature and 'summary' is the label or target variable.

### What does the data look like? ("EDA")

I've printed three intros to news documents out below to illustrate the variety of 'news' that is in this dataset. What are the linguistic features and challenges the model will need to learn?

- "New Zealand is Wild"
  - Almost conversational travel journalism talking to a man with a wild animal attack,
  - Factor in regional terms of speech e.g. UK/AU/NZ English that will be uncommon compared to the mostly American English data body

- Charlie Sheen
  - This looks like it is from an internet message board, forum usernames (e.g. DangerGuerrero) which are highly unique and not likely to be found elsewhere or in any other context.
  - Drug references, sarcasm, urban myths and in-jokes laced with context the model will not explicitly see will challenge it.

- Obama administration vs Fox News
  - Quite matter-of-fact with quotes of people speaking in a professional capacity, argumentative tones.

In [40]:
print(news_small['train'][0]['document'][:750])
print("\n \n")
print("====================")
print("\n \n")
print(news_small['train'][10]['document'][:750])
print("\n \n")
print("====================")
print("\n \n")
print(news_small['train'][24]['document'][:750])

New Zealand is wild. 
 
 Just ask this Kiwi kayaker, Kyle Mulinder, who happened to be in “the wrong place at the wrong time” to get slapped directly in the face by an octopus wielded by a seal. 
 
 Yes, you read that right: a seal had an octopus in its mouth in the midst of a battle royale of marine creatures. The seal emerged from the water flailing around with the octopus, which ended up hitting the unsuspecting kayaker in the face while he was out enjoying the great outdoors. 
 
 “We were just sitting out in the middle of the ocean and then this huge male seal appeared with an octopus and he was thrashing him about for ages,” Mulinder told Yahoo News about the freak encounter with nature. Then came the big moment. “I was like ‘mate, wha

 


 

You know, the medicine Charlie Sheen was taking should probably add "locking hookers in the closet" to its official list of side effects. Danger Guerrero 
 
 DangerGuerrero 
 
 Charlie Sheen flipped out on cocaine with a prostitute and wreck

## Preprocessing with T5 Tokenizer

We'll load the T5 AutoTokenizer and preprocessing from here is to prefix the input so the instructions to the LLM are clear (NLP tasks often require prompting) that it's a summarisation task.

In [41]:
from transformers import AutoTokenizer

# load the tokenizer
checkpoint = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [42]:
prefix = "Summarize: "

In [43]:
def preprocess_function(examples):

    # define input with prefix
    inputs = [prefix + doc for doc in examples["document"]]

    model_inputs = tokenizer(
        inputs,
        max_length=1024,
        truncation=True
        ) # start with token input limit 1024, truncate larger instances, default settings

    labels = tokenizer(
        text_target=examples["summary"],
        max_length=128,
        truncation=True
        ) # default settings, text_target is the argument for tokenizing labels

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

Hugging Face `map` method can apply the preprocessing function to the entire dataset

In [44]:
# Apply preprocessing and remove the raw text columns
tokenized_news_small = news_small.map(preprocess_function,
                                      batched=True,
                                      remove_columns=["document", "summary"])

Map:   0%|          | 0/11243 [00:00<?, ? examples/s]

Map:   0%|          | 0/1405 [00:00<?, ? examples/s]

Map:   0%|          | 0/1405 [00:00<?, ? examples/s]

In [45]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

Re above:

- After text is tokenized and transformed, **model inputs have to be uniform length**. The input to a batch must have the same number of tokens (length) to form a valid tensor / array.
  - This is similar to how rows in 2d Pandas DataFrames need the same number of columns to be represented in a valid structure. The uniform arrays/tensors help the model operate and process batches simultaneously.

- We need to deal with sentences of different token length and this is called *dynamic padding* where individual sentences will just be made up with padding to the upper limit. Truncation cuts over maximum sentences down to the limit.

This is done with DataCollatorForSeq2Seq. We will add parameters later that tell the trainer to ignore the padding characters and not learn from them.


So we will 'dynamically pad' sentence by sentence rather than pad the whole dataset to the maximum length. This helps with processing.

## Evaluation

Using ROUGE score (discovered in pre-reading / research and used widely in text / summary comparison)

In [46]:
import evaluate

rouge = evaluate.load("rouge")

this function passes the predictions and labels to compute to calculate the ROUGE metric (use later after training)

In [47]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id) # labels replacement, so padding tokens are properly ignored when evaluating the Rouge score
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True) # will remove special tokens like pad

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

## Training

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Further steps needed:
- Define the training hyperparameters in Seq2SeqTrainingArguments,
  - only required one is output_dir saves the model to HF
  - Reduce batch size to 4 (16 was default) to avoid memory issues, as well as this being a reduced size dataset already

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="small-multi-news-model",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4, # reduced from 16 to 4 to avoid memory issues
    per_device_eval_batch_size=4, # reduced from 16 to 4 to avoid memory issues
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True, #change to bf16=True for XPU
    push_to_hub=True, # this model will automatically be on HF hub when done
    remove_unused_columns=False
)

- Pass training arguments to Seq2SeqTrainer with the model, dataset, tokenizer collator and function above


In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_news_small["train"], # used tokenized dataset not original news_small
    eval_dataset=tokenized_news_small["test"],  # used tokenized dataset not original news_small
    tokenizer=tokenizer,
    data_collator=data_collator, # data collator is handling padding dynamically (see above)
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
print(tokenized_news_small['train'][0])
# document and summary gone now, only input_ids, attention mask and labels remains! Good

{'input_ids': [12198, 1635, 1737, 10, 368, 5725, 19, 3645, 5, 1142, 987, 48, 31092, 16697, 49, 6, 19454, 10094, 77, 588, 6, 113, 2817, 12, 36, 16, 105, 532, 1786, 286, 44, 8, 1786, 97, 153, 12, 129, 3, 7, 8478, 3138, 1461, 16, 8, 522, 57, 46, 3, 32, 75, 2916, 302, 587, 40, 221, 26, 57, 3, 9, 7042, 5, 2163, 6, 25, 608, 24, 269, 10, 3, 9, 7042, 141, 46, 3, 32, 75, 2916, 302, 16, 165, 4247, 16, 8, 3, 12342, 13, 3, 9, 3392, 11268, 15, 13, 8769, 14231, 5, 37, 7042, 13999, 45, 8, 387, 5731, 173, 53, 300, 28, 8, 3, 32, 75, 2916, 302, 6, 84, 3492, 95, 10849, 8, 1149, 76, 5628, 53, 16697, 49, 16, 8, 522, 298, 3, 88, 47, 91, 5889, 8, 248, 10962, 5, 105, 1326, 130, 131, 3823, 91, 16, 8, 2214, 13, 8, 5431, 11, 258, 48, 1450, 5069, 7042, 4283, 28, 46, 3, 32, 75, 2916, 302, 11, 3, 88, 47, 3, 189, 12380, 53, 376, 81, 21, 3, 2568, 642, 10094, 77, 588, 1219, 15670, 3529, 81, 8, 19866, 6326, 28, 1405, 5, 37, 29, 764, 8, 600, 798, 5, 105, 196, 47, 114, 458, 5058, 6, 125, 131, 2817, 58, 22, 153, 282, 34, 

- train() fine tunes the model

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.995000,2.730360,0.146800,0.046400,0.112100,0.112100,19.000000
2,2.929000,2.698031,0.148000,0.047500,0.113200,0.113200,19.000000
3,2.905900,2.682364,0.149800,0.048300,0.114200,0.114200,19.000000
4,2.903400,2.678337,0.150200,0.049300,0.114600,0.114600,19.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control

TrainOutput(global_step=11244, training_loss=2.956871952801911, metrics={'train_runtime': 1576.5135, 'train_samples_per_second': 28.526, 'train_steps_per_second': 7.132, 'total_flos': 1.2169978140033024e+16, 'train_loss': 2.956871952801911, 'epoch': 4.0})

In [9]:
# Reload the model when the runtime finished.
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("tjjdoherty/small-multi-news-model") # model saved on HF
tokenizer = AutoTokenizer.from_pretrained("tjjdoherty/small-multi-news-model")

config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.7k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

## Interpreting the results table above:

- **Epoch:** There were 4 epochs or 'runs' over the entire training data for the model to learn the documents and their summaries, and learn to generate an appropriate output.

- **Training loss:** error on the training dataset. Shows decrease over the four epochs so the model is better predicting over time.
- **Validation loss:** Performance on the validation dataset. Also decreases modestly over time and these values are not too different from the Training Loss. It's a good sign that the model isn't overfitting.


- **ROUGE** (Recall-Oriented Understudy for Gisting Evaluation) **scores:** ROUGE is scored between 0-1 where 0 is no overlap whatsoever between the model output and the reference summary. 1 indicates a perfect overlap. This is measuring overlap with individual words, consecutive words and sentence level.
  - **Rouge1 Value of 0.15** suggest there is some overlap with the reference summaries but still probably missing key details or struggling to produce more informative sentences. Rouge2 is particularly low.
  - **ROUGE-1 and ROUGE-2 are usually low in summarisation tasks if the dataset is full of long documents or the summary styles vary among the training.** The range of document lengths were vast. There were some extremely long documents and some brief news summary reports. I also noticed in my EDA (see section above) that the style varied a lot:
    - some of the news reports are written publications (very matter of fact),
    - others were editor's picks (informative but with a stylistic edge)
    -  others appear to be news anchor-led reports from television shows (full of addresses to the viewer/reader)
    - Others are pure celebrity gossip with jokes, memes and cultural references the model will have more difficulty recognising.
  
  - Definitions of the ROUGE scores:
    - Rouge1 is the overlap of individual words (unigrams) and between 0.3 and 0.5 is considered fairly good.
    - Rouge2 is the overlap of consecutive words (bigrams) and between 0.15 and 0.3 is seen in good models.
    - Rougel (Rouge-L) is the longest common subsequence between predicted and the human/label summary. RougeLsum (Rouge-Lsum) is specifically for summarisations. Between 0.2 and 0.4 are seen as acceptable.
  - Higher ROUGE isn't always better - multiple ways to intelligently summarise a longer text, a larger ROUGE could represent a largely copy-paste job because ROUGE is still measuring on overlap of words or sequences of words.

- **GEN LEN** is generation length which was **19 tokens**. This is very very short, perhaps almost a sentence in length. I expect the ROUGE scores could be improved by simply prompting a longer summary, even by a small amount.
  - **See below**: The model-generated summaries are far too short compared to the references which are around 315 average token length. This matches up with the average **character length** of around 1200 which we found in some earlier EDA.




In [ ]:
import numpy as np

ref_lengths = [len(tokenizer.encode(summary)) for summary in news_small['train']['summary']]
print("Average training reference summary length:", np.mean(ref_lengths))


Average training reference summary length: 315.212843547096


## Key insights:

- The model is learning from each new epoch, both training and validation loss gradually decreasing and ROUGE is slightly improving over time.
- Does not seem to be overfitting, due to the results above
- ROUGE Scores are quite low and the big gap in summary length probably contributes significantly to this. If the model is only generating a 19 token length summary then there is simple not enough content for it to match with the label summary as the ROUGE score represents it.
  - Rouge1 (0.146) is relatively much higher than Rouge2 (0.046), so the model summary is capturing individual word overlaps at a rate three times higher than consecutive word  pairs. The model is therefore capturing key terms/information but not with fluency.

In [ ]:
# push to HF to save
trainer.push_to_hub()

events.out.tfevents.1729727459.847cd201361a.821.0:   0%|          | 0.00/13.0k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/tjjdoherty/small-multi-news-model/commit/1cb22f93afea5a1dd01fad2edcd8daf2fbffc876', commit_message='End of training', commit_description='', oid='1cb22f93afea5a1dd01fad2edcd8daf2fbffc876', pr_url=None, pr_revision=None, pr_num=None)

## Inference:
### Compare 1) a specific training item and 2) a brand new data point

Let's look at the model's output summary vs a ground-truth summary in the raw dataset

In [49]:
sample = tokenized_news_small['train'][0]
input_ids = sample["input_ids"] # input tokens
label_summary = sample["labels"] # ground truth label

In [58]:
import torch

# using the model to generate a summary
generated_ids = model.generate(
    input_ids=torch.tensor([input_ids]), # input IDs passed to the model
    max_length=256,                    # max length higher for longer summary
    num_beams=4,
    early_stopping=True                 # stops when model predicts end token
)

# decode the generated tokens into text
generated_summary = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

#decode ground-truth label (reference summary)
label_summary_decoded = tokenizer.decode(label_summary, skip_special_tokens=True)

#print out the summaries for comparison
print("Generated Summary:", generated_summary)
print("Reference Summary:", label_summary_decoded)

Generated Summary: – A kayaker in New Zealand was slapped directly in the face by an octopus wielded by a seal in the midst of a battle royale of marine creatures. The seal emerged from the water flailing around with the octopus, which ended up hitting the unsuspecting kayaker in the face while he was out enjoying the great outdoors. "We were just sitting out in the middle of the ocean and then this huge male seal appeared with an octopus and he was thrashing him about for ages," said
Reference Summary: – Take that! We can only imagine that's what a seal was thinking when he slapped a kayaker in the face with an octopus over the weekend. Kyle Mulinder was kayaking off the coast of New Zealand's South Island with a group of GoPro content creators, Time reports, when the seal emerged from the depths and hurled an octopus into his head. "I was, like,'mate, what just happened?'" Mulinder tells Yahoo, adding, "It was weird because


On first glance above, these two summaries look quite similar. The generated summary is quite matter of fact and includes a quote which is prominent in the original news. It doesn't have the dramatisation of the reference summary ("The seal was thinking take that when it slapped a kayaker") but it is accurate with the length it's given, despite the cut-off.

Compare the ROUGE Score:

In [60]:
# Compute ROUGE scores between the generated and reference summaries
result = rouge.compute(predictions=[generated_summary], references=[label_summary_decoded])

# Display the results
print("ROUGE Scores:")
for key, value in result.items():
    print(f"{key}: {value:.4f}")

ROUGE Scores:
rouge1: 0.4000
rouge2: 0.1595
rougeL: 0.2545
rougeLsum: 0.2545


### More context on the ROUGE Score:

- Rouge1 is 0.4 compared to the model average of 0.146

- Rouge2 is 0.15 compared to the model average of 0.046

This individual comparison was much stronger. The same trend of ROUGE 1 around 3 x higher than ROUGE 2 is observed.

### New article to summarise

In [63]:
input_text = """
US actor Ron Ely, best known for playing the role of Tarzan in the 1960s television show of the same name, has died aged 86.
"The world has lost one of the greatest men it has ever known - and I have lost my dad." the actor's daughter, Kirsten Casale Ely, said in an Instagram post.
Tarzan originally aired on NBC television network from 1966 to 1968, during which time the actor broke a number of bones and was reported to have been attacked by animals while performing his own stunts.
After retiring from acting in 2001, Ely became an author and published two mystery novels.
Ely made a brief return to acting for one television film, Expecting Amish, in 2014, where he played an Amish elder.
In the 1980s, he appeared in other hit television programmes including cruise ship-based comedy The Love Boat, as well as Wonder Woman with star Lynda Carter.
Born in Texas in 1938, Ely went on to marry his high school sweetheart in 1959, before divorcing two years later.
He was also known for hosting the Miss America pageant in the early 1980s, where he met his wife Valerie. The couple went on to have three children.
Ely died at his home in Los Alamos in Santa Barbara, California on 29 September.
Announcing the death in an Instagram post on Wednesday, the actor's daughter said: "My father was someone that people called a hero. He was an actor, writer, coach, mentor, family man and leader."
She added: "I knew him as my dad - and what a heaven sent honour that has been. To me, he hung the moon.
"""


In [64]:
# Tokenize the input text
inputs = tokenizer(
    input_text,
    return_tensors="pt",  # Return as PyTorch tensors (change to TensorFlow with `tf` if needed)
    truncation=True,  # Truncate if the input is longer than max length
    padding="max_length",  # Pad to max length if necessary
    max_length=1024  # Max length of the training set
)

now generate output

In [66]:
# Generate a summary (or other output depending on the task)
summary_ids = model.generate(
    inputs["input_ids"],  # Input IDs from the tokenizer
    max_length=300,        # Adjust max length of the generated summary
    num_beams=4,          # Beam search for more diverse results (you can adjust this)
    early_stopping=True   # Stop when the model generates the end token
)

# Decode the generated tokens into text
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print("Generated Summary:\n", summary)


Generated Summary:
 – Ron Ely, best known for playing the role of Tarzan in the 1960s television show of the same name, has died aged 86. "The world has lost one of the greatest men it has ever known - and I have lost my dad," the actor's daughter, Kirsten Casale Ely, said in an Instagram post Wednesday. "The world has lost one of the greatest men it has ever known - and I have lost my dad," Ely's daughter Kirsten Casale Ely said in an Instagram post.


Comments on new summarisation:
- The key information was captured - Death of an actor.
- **This summarisation represents the lost-in-the-middle problem very well!**
  - The model completely ignores the middle of the text, not referring to his career, relationships/marriage whatsoever.
  - The model manages to pick out *Wednesday*, the day his daughter posted the quote which is only mentioned in the *second to last* sentence and not attached directly to the sentence it extracted.
- This summary contained two identical quotes to fill out the summary with a very slight rewording outside of the quote.